In [1]:
import torch
from gensim.models import word2vec, Word2Vec
from gensim.models.doc2vec import Doc2Vec
import gensim
import monpa
import csv
from monpa import utils
from tqdm.auto import tqdm
import pandas as pd
import numpy as np
from scipy.spatial.distance import cosine

+---------------------------------------------------------------------+
  Welcome to MONPA: Multi-Objective NER POS Annotator for Chinese
+---------------------------------------------------------------------+
已找到 model檔。Found model file.


c:\Users\eeb02\.virtualenvs\HW2-PfLazmZH\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Word2Vec

## Preprocessing

In [2]:
news_df = pd.read_csv('./terms_data/mid_news_OK.csv', delimiter=',', encoding='utf-8')
bbs_df = pd.read_csv('./terms_data/mid_bbs_OK.csv', delimiter=',', encoding='utf-8')
forum_df = pd.read_csv('./terms_data/mid_forum_OK.csv', delimiter=',', encoding='utf-8')

In [3]:
term_df = pd.concat([news_df, bbs_df, forum_df], ignore_index=True)
term_df.reset_index(drop=True)

,post_time,content
0,2020-01-01,財政部 內政部 一站式 服務 動產 移轉 登記 搞定 避免 民眾 辦理 動產 買賣 移轉 ...
1,2020-01-01,IMF 美元 全球 外匯 儲備 比重 新高 週二 31 國際貨幣組織 公布 IMF 數據 ...
2,2020-01-01,騰訊集團 34億 美元 收購 環球 音樂 10 股權 外電 報導 騰訊集團 0700 HK...
3,2020-01-01,加幣 漲幅 勇冠 10 瑞信 央行 政策 明年 主要 風險 週二 31 受到 中國 強勁 ...
4,2020-01-01,股盤 收低 中央社 台北 2020年 1月 1日 歐洲 股市 今年 最後 交易日 交易 清...
...,...,...
1230605,2023-03-21,友達 ❷❺ 元以 蘇起 不知 多少 老是 說話
1230606,2023-03-21,長期 投資人 ETF 超額 報酬 建議 巴菲特 2013年 股東信 提出 建議 一般人 不...
1230607,2023-03-21,美國 台積電 美國 設廠 不會 影響 美國 台灣 安全 承諾 晶片 中國 打掉 台灣 之後...
1230608,2023-03-21,快笑死 根本 指標 AI 股票 AI 解說 股市 重大 事件 矽谷銀行 倒閉


In [4]:
try:
    del news_df, bbs_df, forum_df
except:
    pass

- 轉為 List of str

In [5]:
def splitWord(x):
    try:
        x = x.split(' ')
    except:
        x = str(x).split(' ')
    
    return x

In [6]:
term_df['content'] = term_df['content'].apply( lambda x: splitWord(x))
term_df['content'].apply( lambda x: x.pop(0))
term_df['content'].head()

0    [財政部, 內政部, 一站式, 服務, 動產, 移轉, 登記, 搞定, 避免, 民眾, 辦理...
1    [IMF, 美元, 全球, 外匯, 儲備, 比重, 新高, 週二, 31, 國際貨幣組織, ...
2    [騰訊集團, 34億, 美元, 收購, 環球, 音樂, 10, 股權, 外電, 報導, 騰訊...
3    [加幣, 漲幅, 勇冠, 10, 瑞信, 央行, 政策, 明年, 主要, 風險, 週二, 3...
4    [股盤, 收低, 中央社, 台北, 2020年, 1月, 1日, 歐洲, 股市, 今年, 最...
Name: content, dtype: object

In [7]:
# raw_sentences[7312]

- Print GPU status

In [8]:
monpa.use_gpu(True)

def print_gpu_memory():
    gpu_state = torch.cuda.is_available()
    total_m = torch.cuda.get_device_properties(0).total_memory/(1024**2)
    reserved_m = torch.cuda.memory_reserved(0)/(1024**2)
    allocated_m = torch.cuda.memory_allocated(0)/(1024**2)
    free_m = reserved_m-allocated_m
    
    print('GPU state: {:s}\nGPU total memory: {:.2f} MB\nReserve for excution: {:.2f} MB\nExecution allocated: {:.2f} MB\nFree memory: {:.2f} MB'\
        .format('Available' if gpu_state else 'None', total_m, reserved_m, allocated_m, free_m))

print_gpu_memory()

GPU state: Available
GPU total memory: 11263.75 MB
Reserve for excution: 0.00 MB
Execution allocated: 0.00 MB
Free memory: 0.00 MB


In [9]:
# Cut into list of list of words
def word_Tokenize(contents: list) -> list:    
    term_list = list()
    step = 1  # 設定幾篇文章合併
    for index in tqdm(range(0, len(contents), step), desc="Wordcutting process: "):
        if  index % 10000 == 0:
            print_gpu_memory()
            # Clear gpu cache
            # torch.cuda.empty_cache()           
        # sentence_list = list()
        # for i in range(step):   # 將多篇文章合併  
        #     if index+i >= len(contents):      
        #         break;    
        sentence_list = utils.short_sentence(contents[index])
        
        result_cut = monpa.cut_batch(sentence_list) #斷詞 GPU加速method
        # print(result_cut)
        tmp_list = list()
        for terms in result_cut:            
            if len(terms) > 0: 
                term = terms[0].strip() #去除前後多餘空白
                # print(term)
                #若詞長>1 (排除標點符號、單字)，排除英數字
                if(len(term)>1 and len(term)<=8) and (not term.isdigit()):
                    tmp_list.append(term)   
        
        if len(tmp_list) > 2:
            term_list.append(tmp_list)                     
    
    return term_list

In [9]:
print_gpu_memory()
# term_list = raw_sentences
term_list = term_df['content']
# term_list = word_Tokenize(raw_sentences)  # ~3hr40min

GPU state: Available
GPU total memory: 11263.75 MB
Reserve for excution: 0.00 MB
Execution allocated: 0.00 MB
Free memory: 0.00 MB


In [10]:
len(term_list)
term_list[1]

['IMF',
 '美元',
 '全球',
 '外匯',
 '儲備',
 '比重',
 '新高',
 '週二',
 '31',
 '國際貨幣組織',
 '公布',
 'IMF',
 '數據',
 '顯示',
 '今年',
 '美元',
 '全球',
 '外匯',
 '儲備',
 '占比',
 '水平',
 '日圓',
 '佔比',
 '持續',
 '升至',
 '20',
 'BR',
 '根據',
 'IMF',
 '數據',
 '美元',
 '持有',
 '儲備',
 '金額',
 '第',
 '3',
 '季',
 '達到',
 '6.75',
 '美元',
 '整體',
 '占比',
 '61.78',
 '低於',
 '第',
 '2',
 '季',
 '6.79',
 '美元',
 '占比',
 '高於',
 '上季',
 '61.49',
 '2018',
 '以來',
 '占比',
 'BR',
 '另外',
 '日圓',
 '第',
 '3',
 '全球',
 '外匯',
 '儲備',
 '佔比',
 '上升',
 '5.6',
 '高於',
 '上季',
 '5.41',
 '2000',
 '年',
 '新高',
 '人民幣',
 '上升',
 '2.01',
 '2016',
 '年',
 '第',
 '4',
 '季',
 '以來',
 '水平',
 'BR',
 '儘管',
 '美元',
 '日圓',
 '人民幣',
 '外匯',
 '儲備',
 '占比',
 '上季',
 '增加',
 '歐元',
 '占比',
 '下降',
 '20.0',
 '2017',
 '年',
 '第',
 '3',
 '季',
 '以來',
 '英鎊',
 '占比',
 '下降',
 '4.43',
 'BR',
 '報告',
 '看來',
 '美元',
 '依舊',
 '位居',
 '全球',
 '主要',
 '儲備',
 '貨幣',
 '寶座',
 '近年',
 '中國',
 '俄羅斯',
 '央行',
 '努力',
 '削弱',
 '美元',
 '霸權',
 '地位',
 '美元',
 '國家',
 '外匯',
 '儲備',
 '分散',
 '出來',
 'BR',
 'Tempus',
 '交易',
 '部門',
 '副總裁',
 'John',


## Training

In [11]:
# Set model configs
import random
config = {
    'vector_size': 250,
    'window': 6,
    'min_count': 10,
    'n_worker': 4,
    'epochs': 10,
    'seed': random.seed(7311),     
}

In [12]:
# Word2Vec Training
model = Word2Vec(sentences=term_list, vector_size=config['vector_size'], window=config['window'], min_count=config['min_count'], workers=config['n_worker'], epochs=config['epochs'], seed=config['seed'])
model.save('./Word2Vec/models/w2v_x250.model')

## Inference

In [13]:
w2v = Word2Vec.load('./Word2Vec/models/w2v_x250.model')

In [23]:
w2v.wv.most_similar('亞馬遜', topn=20)

[('微軟', 0.7384185791015625),
 ('谷歌', 0.7303401827812195),
 ('沃爾瑪', 0.7271144986152649),
 ('Amazon', 0.6968547105789185),
 ('Meta', 0.6739716529846191),
 ('Alphabet', 0.6320428848266602),
 ('蘋果', 0.6058663129806519),
 ('Amazon.com', 0.6026239991188049),
 ('AMZN-US', 0.6015448570251465),
 ('Salesforce', 0.5936291217803955),
 ('網飛', 0.59291672706604),
 ('亞馬', 0.5921523571014404),
 ('特斯拉', 0.5883162021636963),
 ('Shopify', 0.5848883390426636),
 ('AMZN.US', 0.5843936204910278),
 ('WMT-US', 0.5803993344306946),
 ('eBay', 0.5784863829612732),
 ('Etsy', 0.5702493190765381),
 ('WMT', 0.5666285157203674),
 ('Wayfair', 0.5599676966667175)]

In [17]:
def getDocVec(model, docterms: list, vec_size: int):
        docVec = np.zeros(vec_size)
        for term in docterms:
                try:
                        docVec += model.wv.get_vector(term)
                except:
                        continue
        return docVec/len(docterms)

In [4]:
w2v.wv.get_vector('輝達')

array([-0.3259768 , -0.08963934, -1.4230092 , -0.4584967 , -1.7671663 ,
       -3.1347213 ,  1.1926342 , -0.7425937 , -1.0004174 ,  1.6443976 ,
        0.48187158,  1.9733016 , -1.2037859 , -0.5133991 , -0.76947707,
       -0.6595584 ,  0.6938867 , -0.15623398, -0.8099868 ,  1.1948383 ,
        0.35275993, -0.9776071 ,  1.8301219 ,  3.4803278 , -2.65332   ,
        2.0755057 , -2.1783483 ,  0.6475265 ,  3.4772117 ,  2.0485702 ,
        1.4591857 ,  2.0045247 , -1.3418661 , -1.2149343 ,  0.82482696,
       -1.1021904 ,  2.130905  , -0.2452097 , -1.8463897 , -0.5720847 ,
       -1.0489285 , -0.06834292,  0.2727784 ,  1.1981541 ,  1.3986541 ,
       -1.9536257 , -0.5904816 , -2.1405096 , -0.90886116,  3.7766573 ,
       -1.1811665 , -1.5931613 , -1.7466832 ,  1.173405  , -1.3575096 ,
        0.95159423, -1.4727118 , -1.6144832 ,  0.11106312,  2.7539542 ,
       -0.47196868,  1.0432031 , -0.88143045, -1.3062166 ,  0.5475294 ,
        1.3616416 , -1.9597993 ,  0.01964836,  0.3398393 ,  1.65

In [116]:
f = open('./mid-term/terms_data/2603長榮_tfidf_down.csv', newline='', encoding='utf-8')
terms = csv.reader(f, delimiter=',')
test_list = list()
i=0
for term in terms:
    i+=1
    if i>1:
        test_list.append(term[0])
f.close()

In [19]:
# BVector = getDocVec(w2v, test_list, 200)
wordvec1 = getDocVec(w2v, ['美股'], 200)
wordvec2 = getDocVec(w2v, ['亞馬遜'], 200)
similarity = cosine(wordvec1, wordvec2)
similarity

c:\Users\eeb02\.virtualenvs\HW2-PfLazmZH\lib\site-packages\scipy\spatial\distance.py:622: RuntimeWarning: invalid value encountered in scalar divide
  dist = 1.0 - uv / np.sqrt(uu * vv)


0

# Doc2Vec

## Preprocessing

In [2]:
news_df = pd.read_csv('./terms_data/mid_news_OK.csv', delimiter=',', encoding='utf-8')
bbs_df = pd.read_csv('./terms_data/mid_bbs_OK.csv', delimiter=',', encoding='utf-8')
forum_df = pd.read_csv('./terms_data/mid_forum_OK.csv', delimiter=',', encoding='utf-8')

In [3]:
term_df = pd.concat([news_df, bbs_df, forum_df], ignore_index=True)
term_df.reset_index(drop=True)

,post_time,content
0,2020-01-01,財政部 內政部 一站式 服務 動產 移轉 登記 搞定 避免 民眾 辦理 動產 買賣 移轉 ...
1,2020-01-01,IMF 美元 全球 外匯 儲備 比重 新高 週二 31 國際貨幣組織 公布 IMF 數據 ...
2,2020-01-01,騰訊集團 34億 美元 收購 環球 音樂 10 股權 外電 報導 騰訊集團 0700 HK...
3,2020-01-01,加幣 漲幅 勇冠 10 瑞信 央行 政策 明年 主要 風險 週二 31 受到 中國 強勁 ...
4,2020-01-01,股盤 收低 中央社 台北 2020年 1月 1日 歐洲 股市 今年 最後 交易日 交易 清...
...,...,...
1230605,2023-03-21,友達 ❷❺ 元以 蘇起 不知 多少 老是 說話
1230606,2023-03-21,長期 投資人 ETF 超額 報酬 建議 巴菲特 2013年 股東信 提出 建議 一般人 不...
1230607,2023-03-21,美國 台積電 美國 設廠 不會 影響 美國 台灣 安全 承諾 晶片 中國 打掉 台灣 之後...
1230608,2023-03-21,快笑死 根本 指標 AI 股票 AI 解說 股市 重大 事件 矽谷銀行 倒閉


In [4]:
try:
    del news_df, bbs_df, forum_df
except:
    pass

## Tag documents

- Doc2Vec模型的訓練資料集格式為 [sentence, tag]，使用TaggedDocument來包裝資料

In [5]:
TagDoc = gensim.models.doc2vec.TaggedDocument

def tagTrainSet(sentences):
    train_set = []
    for i, text in enumerate(sentences):
        try:
            word_list = text.split(' ')
        except:
            word_list = str(text).split(' ')

        l = len(word_list)
        word_list[l-1] = word_list[l-1].strip()
        doc = TagDoc(word_list, tags=[i])
        train_set.append(doc)
    
    return train_set

## Training

In [6]:
def train(train_data, vec_size):
    model = Doc2Vec(train_data, min_count=1, window=3, vector_size=vec_size, sample=1e-3, negative=5, workers=4)
    model.train(train_data, total_examples=model.corpus_count, epochs=10)
    return model

In [7]:
trainSet = tagTrainSet(term_df['content'])

for i in range(5):
    print(trainSet[i])

In [8]:
d2v = train(trainSet, 250)

In [ ]:
d2v.save('./Word2Vec/D2V_models/d2v_x250.model')

## Inference